In [254]:
import os
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime
from pandas.api.types import CategoricalDtype
import sqlite3
pd.set_option('display.max_columns', 500)


## WM 2018
Auswertung der Ergebnisse

In [255]:
matches_wm2018_results = pd.DataFrame()
matches_wm2018_pre_results = pd.DataFrame()

matches_wm2018_results = pd.read_csv('football_sources/fifa-world-cup-2018-RussianStandardTime_Results.csv')

matches_wm2018_results = matches_wm2018_results.drop(["Location","Group","Round Number", "Date"], axis=1)

matches_wm2018_results["goals_home"] = 0
matches_wm2018_results["goals_away"] = 0

pd.options.display.max_rows = 70
matches_wm2018_results.head(70)

,Home Team,Away Team,Result,goals_home,goals_away
0,Russia,Saudi Arabia,5 - 0,0,0
1,Egypt,Uruguay,0 - 1,0,0
2,Morocco,Iran,0 - 1,0,0
3,Portugal,Spain,3 - 3,0,0
4,France,Australia,2 - 1,0,0
5,Argentina,Iceland,1 - 1,0,0
6,Peru,Denmark,0 - 1,0,0
7,Croatia,Nigeria,2 - 0,0,0
8,Costa Rica,Serbia,0 - 1,0,0
9,Germany,Mexico,0 - 1,0,0


In [256]:
matches_wm2018_results["Result"] = matches_wm2018_results["Result"].astype(str)

for index, row in matches_wm2018_results.iterrows():
    result =  matches_wm2018_results["Result"][index].split(' - ')
    matches_wm2018_results["goals_home"][index] = result[0]
    matches_wm2018_results["goals_away"][index] = result[1]
    
    
matches_wm2018_results = matches_wm2018_results.drop(["Result"], axis=1)
matches_wm2018_results.head(100)

/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Home Team,Away Team,goals_home,goals_away
0,Russia,Saudi Arabia,5,0
1,Egypt,Uruguay,0,1
2,Morocco,Iran,0,1
3,Portugal,Spain,3,3
4,France,Australia,2,1
5,Argentina,Iceland,1,1
6,Peru,Denmark,0,1
7,Croatia,Nigeria,2,0
8,Costa Rica,Serbia,0,1
9,Germany,Mexico,0,1


In [257]:
matches_wm2018_predicted_groups = pd.read_csv('kicktipp_prediction/results_groups.csv', sep=';')
matches_wm2018_predicted_last16 = pd.read_csv('kicktipp_prediction/results_last16.csv')
matches_wm2018_predicted_last8 = pd.read_csv('kicktipp_prediction/results_last8.csv')
matches_wm2018_predicted_last4 = pd.read_csv('kicktipp_prediction/results_last4.csv')
matches_wm2018_predicted_finals = pd.read_csv('kicktipp_prediction/results_finals.csv')

In [258]:
matches_wm2018_predicted_groups = matches_wm2018_predicted_groups[["goals_home","goals_away"]]
matches_wm2018_predicted_last16 = matches_wm2018_predicted_last16[["goals_home","goals_away"]]
matches_wm2018_predicted_last8 = matches_wm2018_predicted_last8[["goals_home","goals_away"]]
matches_wm2018_predicted_last4 = matches_wm2018_predicted_last4[["goals_home","goals_away"]]
matches_wm2018_predicted_finals = matches_wm2018_predicted_finals[["goals_home","goals_away"]]

In [259]:
matches_wm2018_predicted = pd.DataFrame()

matches_wm2018_predicted = matches_wm2018_predicted_groups
matches_wm2018_predicted = matches_wm2018_predicted.append(matches_wm2018_predicted_last16)
matches_wm2018_predicted = matches_wm2018_predicted.append(matches_wm2018_predicted_last8)
matches_wm2018_predicted = matches_wm2018_predicted.append(matches_wm2018_predicted_last4)
matches_wm2018_predicted = matches_wm2018_predicted.append(matches_wm2018_predicted_finals)


matches_wm2018_predicted

,goals_home,goals_away
0,7.0,6.0
1,6.0,6.0
2,5.0,3.0
3,7.0,6.0
4,6.0,5.0
5,6.0,6.0
6,3.0,8.0
7,6.0,5.0
8,6.0,5.0
9,2.0,7.0


In [260]:
matches_wm2018_predicted = matches_wm2018_predicted.reset_index()
matches_wm2018_predicted = matches_wm2018_predicted.drop(["index"], axis=1)
matches_wm2018_predicted

,goals_home,goals_away
0,7.0,6.0
1,6.0,6.0
2,5.0,3.0
3,7.0,6.0
4,6.0,5.0
5,6.0,6.0
6,3.0,8.0
7,6.0,5.0
8,6.0,5.0
9,2.0,7.0


In [261]:
for index in range(0,48):
        
    if matches_wm2018_predicted["goals_home"][index] > matches_wm2018_predicted["goals_away"][index]:
        
        #neccessary vars
        difference = (matches_wm2018_predicted["goals_home"][index] - matches_wm2018_predicted["goals_away"][index])
    
        matches_wm2018_predicted["goals_away"][index] = 1
        matches_wm2018_predicted["goals_home"][index] = matches_wm2018_predicted["goals_away"][index] + difference
              
            
    if matches_wm2018_predicted["goals_away"][index] > matches_wm2018_predicted["goals_home"][index]:
        
        #neccessary vars
        difference = (matches_wm2018_predicted["goals_away"][index] - matches_wm2018_predicted["goals_home"][index])
    
        matches_wm2018_predicted["goals_home"][index] = 1
        matches_wm2018_predicted["goals_away"][index] = matches_wm2018_predicted["goals_home"][index] + difference
        
    if matches_wm2018_predicted["goals_away"][index] == matches_wm2018_predicted["goals_home"][index]:
        matches_wm2018_predicted["goals_home"][index] = 1
        matches_wm2018_predicted["goals_away"][index] = 1
        
        
for index in range(48,64):
    
    if matches_wm2018_predicted["goals_home"][index] > matches_wm2018_predicted["goals_away"][index]:
    
        if (matches_wm2018_predicted["goals_home"][index] + matches_wm2018_predicted["goals_away"][index])>=11:
            matches_wm2018_predicted["goals_home"][index] = 2
            matches_wm2018_predicted["goals_away"][index] = 1
            
        elif (matches_wm2018_predicted["goals_home"][index] + matches_wm2018_predicted["goals_away"][index])<11:
            matches_wm2018_predicted["goals_home"][index] = 1
            matches_wm2018_predicted["goals_away"][index] = 0
        
    if matches_wm2018_predicted["goals_away"][index] > matches_wm2018_predicted["goals_home"][index]:
        
        if (matches_wm2018_predicted["goals_home"][index] + matches_wm2018_predicted["goals_away"][index])>=11:
            matches_wm2018_predicted["goals_home"][index] = 1
            matches_wm2018_predicted["goals_away"][index] = 2
            
        elif (matches_wm2018_predicted["goals_home"][index] + matches_wm2018_predicted["goals_away"][index])<11:
            matches_wm2018_predicted["goals_home"][index] = 0
            matches_wm2018_predicted["goals_away"][index] = 1
        
matches_wm2018_predicted

,goals_home,goals_away
0,2.0,1.0
1,1.0,1.0
2,3.0,1.0
3,2.0,1.0
4,2.0,1.0
5,1.0,1.0
6,1.0,6.0
7,2.0,1.0
8,2.0,1.0
9,1.0,6.0


In [262]:
matches_wm2018_predicted["goals_home"][0] == matches_wm2018_results["goals_home"][0]

False

In [263]:
matches_wm2018_predicted["goals_home"][43] = 1
matches_wm2018_predicted["goals_home"][0] = 0
matches_wm2018_predicted["goals_home"][9] = 6
matches_wm2018_predicted["goals_away"][9] = 1

In [264]:
matches_wm2018_predicted["score_for_this_game"] = 0

for index, row in matches_wm2018_results.iterrows():
    
    difference_real = matches_wm2018_results["goals_home"][index] - matches_wm2018_results["goals_away"][index]
    difference_pred = matches_wm2018_predicted["goals_home"][index] - matches_wm2018_predicted["goals_away"][index]
    points = 0
    
    if matches_wm2018_predicted["goals_home"][index] > matches_wm2018_predicted["goals_away"][index] and matches_wm2018_results["goals_home"][index] > matches_wm2018_results["goals_away"][index]:
        points = points + 2
        
        if (difference_real == difference_pred):
            points = points + 1
    
        if (matches_wm2018_predicted["goals_home"][index] == matches_wm2018_results["goals_home"][index]) and (matches_wm2018_predicted["goals_away"][index] == matches_wm2018_results["goals_away"][index]):  
            points = points + 1
        
    if matches_wm2018_predicted["goals_home"][index] < matches_wm2018_predicted["goals_away"][index] and (matches_wm2018_results["goals_home"][index] < matches_wm2018_results["goals_away"][index]):
        points = points + 2
        
        if (difference_real == difference_pred):
            points = points + 1
    
        if (matches_wm2018_predicted["goals_home"][index] == matches_wm2018_results["goals_home"][index]) and (matches_wm2018_predicted["goals_away"][index] == matches_wm2018_results["goals_away"][index]):  
            points = points + 1
        
    if matches_wm2018_predicted["goals_home"][index] == matches_wm2018_predicted["goals_away"][index] and (matches_wm2018_results["goals_home"][index] == matches_wm2018_results["goals_away"][index]):
        points = points + 2
        
        if (matches_wm2018_predicted["goals_home"][index] == matches_wm2018_results["goals_home"][index]) and (matches_wm2018_predicted["goals_away"][index] == matches_wm2018_results["goals_away"][index]):  
            points = points + 2
        

        
    
    matches_wm2018_predicted["score_for_this_game"][index] = points
        
        
matches_wm2018_predicted

/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,goals_home,goals_away,score_for_this_game
0,0.0,1.0,0
1,1.0,1.0,0
2,3.0,1.0,0
3,2.0,1.0,0
4,2.0,1.0,4
5,1.0,1.0,4
6,1.0,6.0,2
7,2.0,1.0,2
8,2.0,1.0,0
9,6.0,1.0,0


In [269]:
matches_wm2018_analyse = matches_wm2018_results
matches_wm2018_analyse

,Home Team,Away Team,goals_home,goals_away
0,Russia,Saudi Arabia,5,0
1,Egypt,Uruguay,0,1
2,Morocco,Iran,0,1
3,Portugal,Spain,3,3
4,France,Australia,2,1
5,Argentina,Iceland,1,1
6,Peru,Denmark,0,1
7,Croatia,Nigeria,2,0
8,Costa Rica,Serbia,0,1
9,Germany,Mexico,0,1


In [270]:
matches_wm2018_analyse["goals_home_pred"] = 0
matches_wm2018_analyse["goals_away_pred"] = 0
matches_wm2018_analyse["goals_home_actual"] = 0
matches_wm2018_analyse["goals_away_actual"] = 0
matches_wm2018_analyse["score"] = 0

for index, row in matches_wm2018_analyse.iterrows():
    matches_wm2018_analyse["goals_home_pred"][index] = matches_wm2018_predicted["goals_home"][index]
    matches_wm2018_analyse["goals_away_pred"][index] = matches_wm2018_predicted["goals_away"][index]
    matches_wm2018_analyse["goals_home_actual"][index] = matches_wm2018_analyse["goals_home"][index]
    matches_wm2018_analyse["goals_away_actual"][index] = matches_wm2018_analyse["goals_away"][index]
    matches_wm2018_analyse["score"][index] = matches_wm2018_predicted["score_for_this_game"][index]
    
matches_wm2018_analyse = matches_wm2018_analyse.drop(["goals_home","goals_away"], axis=1)

matches_wm2018_analyse

/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/marco/anaconda3/lib/python3.6/site-packa

,Home Team,Away Team,goals_home_pred,goals_away_pred,goals_home_actual,goals_away_actual,score
0,Russia,Saudi Arabia,0,1,5,0,0
1,Egypt,Uruguay,1,1,0,1,0
2,Morocco,Iran,3,1,0,1,0
3,Portugal,Spain,2,1,3,3,0
4,France,Australia,2,1,2,1,4
5,Argentina,Iceland,1,1,1,1,4
6,Peru,Denmark,1,6,0,1,2
7,Croatia,Nigeria,2,1,2,0,2
8,Costa Rica,Serbia,2,1,0,1,0
9,Germany,Mexico,6,1,0,1,0


In [271]:
matches_wm2018_analyse.to_csv("matches_analysis.csv", encoding='utf-8')